In [3]:
import requests

from api_key import api_key

In [25]:
url = "https://api.um.warszawa.pl/api/action/busestrams_get"

In [26]:
params = {
    "resource_id": "f2e5503e-927d-4ad3-9500-4ab9e55deb59",  # resource id given by the API
    "apikey": api_key,
    "type": 1,  # 1 - bus, 2 - tram
}

In [27]:
response = requests.get(url, params=params)

In [18]:
print(response.json())

{'result': [{'Lines': '219', 'Lon': 21.115034, 'VehicleNumber': '1000', 'Time': '2024-12-25 16:52:05', 'Lat': 52.234593, 'Brigade': '1'}, {'Lines': '225', 'Lon': 21.115663, 'VehicleNumber': '1001', 'Time': '2024-12-24 08:14:35', 'Lat': 52.234589, 'Brigade': '503'}, {'Lines': '213', 'Lon': 21.115089, 'VehicleNumber': '1002', 'Time': '2024-12-26 02:44:21', 'Lat': 52.234708, 'Brigade': '5'}, {'Lines': '213', 'Lon': 21.115123, 'VehicleNumber': '1003', 'Time': '2024-12-24 23:38:49', 'Lat': 52.234581, 'Brigade': '3'}, {'Lines': '219', 'Lon': 21.115253, 'VehicleNumber': '1004', 'Time': '2024-12-23 22:45:50', 'Lat': 52.234574, 'Brigade': '3'}, {'Lines': '213', 'Lon': 21.115905, 'VehicleNumber': '1005', 'Time': '2024-12-26 03:00:10', 'Lat': 52.234286, 'Brigade': '1'}, {'Lines': '311', 'Lon': 21.11533, 'VehicleNumber': '1006', 'Time': '2024-12-23 21:50:21', 'Lat': 52.234588, 'Brigade': '502'}, {'Lines': '213', 'Lon': 21.17877, 'VehicleNumber': '1007', 'Time': '2024-12-26 14:24:44', 'Lat': 52.184

In [19]:
print(len(response.json()["result"]))

1896
